In [48]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import csv
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime

# EDA

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train_X = train.drop(labels = 'target', axis = 1)
train_y = train.drop(labels = ['id','keyword','location','text'], axis = 1)

In [4]:
print("Dimention of train data : ",train_X.shape)
print("Dimention of train label : ",train_y.shape)
print("Dimention of test data : ",test.shape)

Dimention of train data :  (7613, 4)
Dimention of train label :  (7613, 1)
Dimention of test data :  (3263, 4)


In [5]:
#Null Data Field
for c in train_X.columns:   
    print("Number of Null Cells in ---",c,"----are ",train_X[c].isnull().sum())

Number of Null Cells in --- id ----are  0
Number of Null Cells in --- keyword ----are  61
Number of Null Cells in --- location ----are  2533
Number of Null Cells in --- text ----are  0


In [6]:
Total_data = pd.concat([train_X, test], ignore_index=True)

In [7]:
Total_data

,id,keyword,location,text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...
...,...,...,...,...
10871,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
10872,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
10873,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
10874,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [8]:
type(text)

NameError: name 'text' is not defined

# PreProcessing

In [38]:
def tags(data):
    pattern = re.compile(r'[@|#][a-zA-Z]+')
    matches = pattern.findall(data)
    tags = [match[1:].lower() for match in matches]
    return ' '.join(tags)

def striplinks(data):
    cleanr = re.compile(r'(?:(?:http|https|ftp):\/\/)[\w/\-?=%.]+\.[\w/\-?=%.]+')
    clean = re.sub(cleanr, ' ', data)
    return clean
    
def alphanumeric(data):
    cleanr = re.compile(r'[^A-Za-z0-9]+')
    clean = re.sub(cleanr, ' ', data)
    return clean


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [10]:
stop_words = set(stopwords.words('english'))
stemmer  = SnowballStemmer('english')

In [51]:
start = datetime.now()
with open('processed_total_data.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['tweets'])
    for i in tqdm(range(10876)):
        
        #raw data
        keyword = str(Total_data['keyword'][i])
        location = str(Total_data['location'][i])
        text = str(Total_data['text'][i])
   
        #extract tags 
        tags_str = stemmer.stem(tags(text))
        #print(tags_str)
        #strip links and alphanumberic charcters
        text_sl = striplinks(text)
        text_sl_an = alphanumeric(text_sl)
        #print(text_sl)
        #print(text_sl_an)
        #removing stopwords and stem the text       
        text_words = word_tokenize(text_sl_an.lower())
        text_sl_an_sw_st = ' '.join(stemmer.stem(j) for j in text_words if j not in stop_words )
        #print(text_words)
        #print(text_sl_an_sw_st)
        #assigning weights
        processed_list = []
        if(tags_str != ''):
            processed_list.extend([tags_str,tags_str,tags_str,text_sl_an_sw_st])
        else:
            processed_list.extend([text_sl_an_sw_st])
        if (keyword != 'nan'):
            #print(keyword)
            processed_list.extend([keyword,keyword])
        if (location != 'nan'):
            location_words = word_tokenize(alphanumeric(location))
            location_str = " ".join(location_words)
            processed_list.extend([location_str])
        
        #saving string
        
        processed_string = " ".join(processed_list)
        #print(i, processed_string)
        writer.writerow([processed_string])
print("Time to run this subsection :", datetime.now() - start)

100%|██████████████████████████████████████████████████████████████████████████| 10876/10876 [00:05<00:00, 1931.06it/s]


Time to run this subsection : 0:00:05.638123


In [ ]:
Total_data['keyword'][0] == float('nan')

In [ ]:
str(Total_data['keyword'][0]) == 'nans'

In [22]:
stemmer.stem("painting is funny thing swimming")

'painting is funny thing swim'

In [26]:
print(stop_words)

{"weren't", 'such', "she's", 'these', 'if', 'those', 'and', 'here', 'should', 'its', 'itself', 'i', 'll', 'ours', 'then', 'ma', 'further', 'to', "haven't", 't', 'each', 'hadn', 'my', 'while', 'she', 'into', 'him', "you're", 'me', 'so', "hadn't", "isn't", 'd', 'out', 'myself', 'do', 'himself', "aren't", 'ourselves', 'when', 'why', 'in', 'too', 'doing', 'until', "you'd", "shouldn't", 'isn', 'than', 're', 'her', 'not', "couldn't", 'are', "it's", 'be', 'it', 'just', "needn't", 'yourselves', 'had', 'below', 'their', 'up', 'your', "that'll", 'aren', 'was', 'm', 'only', 'or', 'that', 'did', 'a', "wasn't", "you've", 'by', 'between', 'more', 'we', 'being', 'same', 'herself', 'am', 'down', 'don', 'y', 'wouldn', 'no', 's', 'against', 'very', 'any', 'theirs', 'how', 'ain', 'through', 'hasn', "mightn't", 'haven', 'all', 'which', 'other', "you'll", 'at', 'them', "hasn't", 'an', 'does', 'now', 'his', 'some', 'themselves', 'whom', 'once', 'shouldn', 'will', 'most', 'couldn', 'been', 'the', 'but', 'own

In [41]:
string = "nokia, asd;asnkxa , INdia"
word_tokenize(string)


['nokia', ',', 'asd', ';', 'asnkxa', ',', 'INdia']

In [46]:
alphanumeric("Gloucestershire,UK")


'Gloucestershire UK'